In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import json
import pymysql

### 한국은행 OpenAPI 조회 서비스 사용 
http://ecos.bok.or.kr/jsp/openapi/OpenApiController.jsp?t=main

In [2]:
"""
한국은행 openAPI를 활용하여
소비자동향조사 받아옴
"""
def getEcosAPI(API_KEY, code, max, start_month, end_month):
    url = 'http://ecos.bok.or.kr/api/StatisticSearch/%s/json/kr/1/%s/%s/MM/%s/%s/?/?/?/' % (API_KEY, max, code, start_month, end_month)
    result = urlopen(url)
    html = result.read()
    
    return json.loads(html)


In [3]:
API_KEY = "YOUR_API_KEY"

data = getEcosAPI(API_KEY, code = '040Y002', max = 100000, start_month = '200801', end_month = '202010')

### 소비자심리지수 구하기
ITEM_CODE1 = 'FME'

In [4]:
data = data["StatisticSearch"]["row"]
produce = pd.DataFrame(data)

In [5]:
ccsi = produce[produce['ITEM_CODE1'] == 'FME']
ccsi

,UNIT_NAME,STAT_NAME,ITEM_CODE1,STAT_CODE,ITEM_CODE2,ITEM_CODE3,ITEM_NAME1,ITEM_NAME2,DATA_VALUE,ITEM_NAME3,TIME
79992,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,73.7,,200807
79993,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,91.8,,200808
79994,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,90.6,,200809
79995,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,77.9,,200810
79996,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,72.9,,200811
...,...,...,...,...,...,...,...,...,...,...,...
80134,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,77.6,,202005
80135,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,81.8,,202006
80136,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,84.2,,202007
80137,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,88.2,,202008


In [6]:
ccsi = ccsi.loc[:, ['TIME', 'DATA_VALUE']].reset_index(drop=True)
ccsi

,TIME,DATA_VALUE
0,200807,73.7
1,200808,91.8
2,200809,90.6
3,200810,77.9
4,200811,72.9
...,...,...
142,202005,77.6
143,202006,81.8
144,202007,84.2
145,202008,88.2


### DB연동

In [16]:
conn = pymysql.connect(host = "127.0.0.1", user = [USER], passwd = [PASSWORD], db = [DATABASE])
cur = conn.cursor()

### DB table 생성

In [17]:
"""
CCSI 저장할 table 생성
"""

create_table_query = """
CREATE TABLE IF NOT EXISTS ccsi(
    id BIGINT(7) NOT NULL AUTO_INCREMENT,
    year bigint(4) NOT NULL,
    month bigint(2) NOT NULL,
    ccsi double,
    primary key(id) )
    charset=utf8mb4;
"""
cur.execute(create_table_query)
# charset=utf8mb4; 유니코드 스트링 저장

0

In [18]:
cur.execute("desc ccsi")
table = cur.fetchall()
pd.DataFrame(list(table))

,0,1,2,3,4,5
0,id,bigint(7),NO,PRI,None,auto_increment
1,year,bigint(4),NO,,None,
2,month,bigint(2),NO,,None,
3,ccsi,double,YES,,None,


### CCSI 월별로 저장

In [19]:
"""
날짜에서 year, month 추출
"""
def extract_year_and_month(period):
    year = period[0:4]
    month = period[4:6]
    return year, month


In [20]:
"""
db table에 저장
"""
for i in range(len(ccsi)):
    query = """ 
    Insert into ccsi (year, month, ccsi) values (%d, %d, %f) ;
    """
    year, month = extract_year_and_month(ccsi.iloc[i].TIME)
    ccsi_value = ccsi.iloc[i].DATA_VALUE

    mystring = (query % (int(year), int(month), float(ccsi_value)))
    print(mystring)
    cur.execute(mystring)

 
    Insert into ccsi (year, month, ccsi) values (2008, 7, 73.700000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2008, 8, 91.800000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2008, 9, 90.600000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2008, 10, 77.900000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2008, 11, 72.900000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2008, 12, 67.700000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2009, 1, 74.800000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2009, 2, 76.400000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2009, 3, 72.800000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2009, 4, 93.000000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2009, 5, 104.100000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2009, 6, 106.600000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2009, 7,

In [21]:
# 확인
query = """ 
select * from ccsi ;
"""
cur.execute(query)
tmp = pd.DataFrame(list(cur.fetchall()))
tmp

,0,1,2,3
0,1,2008,7,73.7
1,2,2008,8,91.8
2,3,2008,9,90.6
3,4,2008,10,77.9
4,5,2008,11,72.9
...,...,...,...,...
142,143,2020,5,77.6
143,144,2020,6,81.8
144,145,2020,7,84.2
145,146,2020,8,88.2


In [22]:
conn.commit()
cur.close()
conn.close()